In [ ]:
# pip list

In [1]:
import torch


In [2]:
if torch.cuda.is_available():
    print('gpu available')
    print(torch.cuda.get_device_name())
    device = torch.device('cuda')
else:
    print('gpu not available')
    device = torch.device('cpu')

gpu available
NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
# !pip install transformers

In [ ]:
# !pip install sentencepiece

In [4]:
#!pip install pandas

In [5]:
import pandas as pd

In [6]:
df= pd.read_csv('train.csv',header=None)
df.columns = ['text','mask','label']

In [7]:
df.head()

,text,mask,label
0,BOS Sir I created a new env with python 3.7 bu...,O O O O O O O O O O O O O O O O O O O O O O O ...,need_help
1,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help
2,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings
3,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great
4,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link


In [8]:
df.shape

(3514, 3)

In [9]:
possible_lables = df['label'].unique()

In [10]:
lable_dict = {}
for index,possible_lable in enumerate(possible_lables):
    lable_dict[possible_lable] = index
    
    
df['label_ecoded']= df['label'].replace(lable_dict)
df.head()

,text,mask,label,label_ecoded
0,BOS Sir I created a new env with python 3.7 bu...,O O O O O O O O O O O O O O O O O O O O O O O ...,need_help,0
1,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help,0
2,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings,1
3,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great,2
4,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link,3


In [11]:
n =len(possible_lables)

In [12]:
n

151

In [13]:
df['label_ecoded'].value_counts()

label_ecoded
0      781
8      609
1      331
13     284
27     169
      ... 
112      1
113      1
115      1
116      1
150      1
Name: count, Length: 151, dtype: int64

In [14]:
sents = df['text'].values
labels = df['label_ecoded'].values

In [15]:
from transformers import AlbertTokenizer

print('loading albert tokenizer.........')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2',do_lower_case =True)

loading albert tokenizer.........


In [16]:
tokenizer

AlbertTokenizer(name_or_path='albert-base-v2', vocab_size=30000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [17]:
max_len = 0

for sent in sents:
    input_ids = tokenizer.encode(sent,add_special_tokens=True)
    
    max_len = max(max_len,len(input_ids))

print(f'Max length of Snetence is : {max_len}')

Token indices sequence length is longer than the specified maximum sequence length for this model (634 > 512). Running this sequence through the model will result in indexing errors


Max length of Snetence is : 738


In [20]:
input_ids = []
attention_mask = []

for sent in sents:
    
    encoder_dict = tokenizer.encode_plus(sent,add_special_tokens=True,max_length=max_len,
                                        return_attention_mask=True,
                                        return_tensors='pt',
                                        truncation=True)
    input_ids.append(encoder_dict['input_ids'])
    attention_mask.append(encoder_dict['attention_mask'])
    
    
    
input_ids = torch.cat(input_ids,dim=0)
attention_mask = attention_mask.cat(input_ids,dim=0)
labels = torch.tensor(labels)


print(f'original sent : {sents[0]}')
print(f'input IDS : {input_ids[0]}')



RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 54 but got size 15 for tensor number 1 in the list.

In [ ]:
#import torch
#from transformers import AlbertTokenizer

input_ids = []
attention_mask = []

for sent in sents:
    encoder_dict = tokenizer.encode_plus(
        sent,
        add_special_tokens=True,
        max_length=100,
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True,
        padding='longest',
    )
    input_ids.append(encoder_dict['input_ids'])
    attention_mask.append(encoder_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_mask = torch.cat(attention_mask, dim=0)
labels = torch.tensor(labels)

print(f'original sent : {sents[0]}')
print(f'input IDS : {input_ids[0]}')


In [ ]:
import torch
from transformers import AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2',do_lower_case =True)

max_len = 0

for sent in sents:
    input_ids = tokenizer.encode(sent,add_special_tokens=True)
    
    max_len = max(max_len,len(input_ids))

print(f'Max length of Snetence is : {max_len}')

input_ids = []
attention_mask = []

for sent in sents:
    
    encoder_dict = tokenizer.encode_plus(sent,add_special_tokens=True,max_length=max_len,
                                        return_attention_mask=True,
                                        return_tensors='pt',
                                        truncation=True)
    input_ids.append(encoder_dict['input_ids'])
    attention_mask.append(encoder_dict['attention_mask'])
    
    
    
input_ids = torch.cat(input_ids,dim=0)
attention_mask = torch.cat(attention_mask,dim=1)
labels = torch.tensor(labels)


print(f'original sent : {sents[0]}')
print(f'input IDS : {input_ids[0]}')


In [ ]:
import torch
import pandas as pd
from transformers import AlbertTokenizer

# Load and preprocess the data
df = pd.read_csv('train.csv', header=None)
df.columns = ['text', 'mask', 'label']

possible_labels = df['label'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['label_encoded'] = df['label'].replace(label_dict)

sents = df['text'].values
labels = df['label_encoded'].values

# Initialize the tokenizer
print('Loading ALBERT tokenizer...')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)

max_len = 0

for sent in sents:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print(f'Max length of sentence is: {max_len}')

input_ids = []
attention_mask = []

for sent in sents:
    encoder_dict = tokenizer.encode_plus(
        sent,
        add_special_tokens=True,
        max_length=100,
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    input_ids.append(encoder_dict['input_ids'])
    attention_mask.append(encoder_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_mask = torch.cat(attention_mask, dim=0)
labels = torch.tensor(labels)

print(f'Original sentence: {sents[0]}')
print(f'Input IDs: {input_ids[0]}')


In [22]:
import torch
import pandas as pd
from transformers import AlbertTokenizer

# Load and preprocess the data
df = pd.read_csv('train.csv', header=None)
df.columns = ['text', 'mask', 'label']

possible_labels = df['label'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['label_encoded'] = df['label'].replace(label_dict)

sents = df['text'].values
labels = df['label_encoded'].values

# Initialize the tokenizer
print('Loading ALBERT tokenizer...')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)

max_len = 0
input_ids = []
attention_mask = []

for sent in sents:
    encoder_dict = tokenizer.encode_plus(
        sent,
        add_special_tokens=True,
        max_length=100,
        return_attention_mask=True,
        truncation=True
    )
    input_ids.append(encoder_dict['input_ids'])
    attention_mask.append(encoder_dict['attention_mask'])

max_len = max(len(ids) for ids in input_ids)

# Pad the input_ids and attention_mask tensors
input_ids = [ids + [tokenizer.pad_token_id] * (max_len - len(ids)) for ids in input_ids]
attention_mask = [mask + [0] * (max_len - len(mask)) for mask in attention_mask]

input_ids = torch.tensor(input_ids)
attention_mask = torch.tensor(attention_mask)
labels = torch.tensor(labels)

print(f'Original sentence: {sents[20]}')
print(f'Input IDs: {input_ids[20]}')


Loading ALBERT tokenizer...
Original sentence: BOS how to open terminal from the project folder to execute the git command EOS
Input IDs: tensor([    2, 11054,   184,    20,   368,  3855,    37,    14,   669, 19294,
           20, 15644,    14,    13, 10404,  1202,    13,  3894,    18,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
